> ### Url & Filters Inputs

In [1]:
url ="https://www.amazon.com/dp/B07XWGWPH5?th=1"

filter_type_list = {'all reviews': 0, 'varified purchase': 1}
filtertype = filter_type_list['all reviews']

filter_rating_list = {'5 stars': 1,'4 stars': 2,'3 stars': 3,'2 stars': 4,'1 star': 5,'positive': 6,'critical': 7}
stars = filter_rating_list['1 star']

filter_version_list = {'all': 0, 'version: Blue Quantum |Style: N200': 1}
version = filter_version_list['version: Blue Quantum |Style: N200']

filter_formate_list = {'all': 0, 'non text': 1}
formate = filter_formate_list['all']

> ### Dependancies Cell

In [2]:
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import Firefox
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.events import EventFiringWebDriver, AbstractEventListener
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.options import Options
Firefox_options = Options()
#Firefox_options.add_argument("--disable-extensions")
Firefox_options.add_argument("--disable-gpu")
Firefox_options.add_argument("--headless")
Firefox_options.add_argument("--start-maximized")
# Firefox_options.headless = True # also works
import time
from time import sleep
import pandas as pd
import numpy as np


> ### Reviews Scraper

In [3]:
driver = webdriver.Firefox(options=Firefox_options)
# driver = webdriver.Firefox()
actions = ActionChains(driver)
driver.get(url)

# see all reviews
all_reviews = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.LINK_TEXT, 'See all reviews')))
all_reviews.click()

# purchase varification
filter_type = driver.find_element(By.ID, 'a-autoid-4-announce')
filter_type.click()
filter_type_ = driver.find_element(
    By.ID, f'reviewer-type-dropdown_{filtertype}')
filter_type_.click()

# reviews rating
filter_ratineg = driver.find_element(By.ID, 'a-autoid-5-announce')
filter_ratineg.click()
filter_ratineg_ = driver.find_element(By.ID, f'star-count-dropdown_{stars}')
filter_ratineg_.click()

# reveiwed version
filter_version = driver.find_element(By.ID, 'a-autoid-6-announce')
filter_version.click()
filter_version_ = driver.find_element(By.ID, f'format-type-dropdown_{version}')
filter_version_.click()

# reveiw media (text or non text)
filter_formate = driver.find_element(By.ID, 'a-autoid-7-announce')
filter_formate.click()
filter_formate_ = driver.find_element(By.ID, f'media-type-dropdown_{formate}')
filter_formate_.click()

result_list = []
while True:

    page_loaded = False
    while page_loaded == False:
        try:
            sleep(0.02)
            results_container = driver.find_element(By.ID, 'cm_cr-review_list')
            results = results_container.find_elements(
                By.CSS_SELECTOR, 'div[class="a-section celwidget"]')
            end_of_the_page = driver.find_element(By.ID, 'sis_pixel_r2')
            page_loaded = True
        except:
            page_loaded = False

    for R in results:
        # customer's name
        name = R.find_element(By.TAG_NAME, 'div').text

        # Review Rating
        rating_E = R.find_element(
            By.CLASS_NAME, 'a-icon-alt').get_attribute('innerHTML')
        rating = rating_E[:rating_E.index(" ")]

        # rewiew title
        review_title_E = R.find_element(
            By.CSS_SELECTOR, 'a[data-hook="review-title"]').get_attribute('innerHTML')
        review_title = review_title_E[review_title_E.index(
            ">")+1:review_title_E.index('</')]

        # date and country of the review
        T_and_P = R.find_element(
            By.CSS_SELECTOR, 'span[data-hook="review-date"]').get_attribute('innerHTML').split(" on ")
        Time = T_and_P[1]
        Place = (T_and_P[0])[(T_and_P[0]).index('in ')+3:]

        # reiw comment
        comment = R.find_element(
            By.CSS_SELECTOR, 'span[class="a-size-base review-text review-text-content"]').text

        # purchase varification
        try:
            varification = R.find_element(
                By.CSS_SELECTOR, 'span[data-hook="avp-badge"]').text.split(" ")[0]
        except:
            varification = 'not varified'

        review = {"Customer's Name": name, "Review Rating": rating, "Rewiew Title": review_title,
                  "Review Country": Place, "Review Date": Time, "Comment": comment,
                  "purchase Type": varification}
        result_list.append(review)

        # next page element
        next_page_E = driver.find_element(By.ID, 'cm_cr-pagination_bar')
        next_page = next_page_E.find_element(By.CLASS_NAME, 'a-last')
        
    try:
        next = next_page.find_element(By.TAG_NAME, 'a')# insures that the next page button has a link and activate
        next_page.click()
        driver.switch_to.default_content()
    except:
        break
    sleep(1)


main_df = pd.DataFrame(result_list)
driver.quit()
print("done")


done


In [4]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Customer's Name  83 non-null     object
 1   Review Rating    83 non-null     object
 2   Rewiew Title     83 non-null     object
 3   Review Country   83 non-null     object
 4   Review Date      83 non-null     object
 5   Comment          83 non-null     object
 6   purchase Type    83 non-null     object
dtypes: object(7)
memory usage: 4.7+ KB


In [5]:
main_df.tail()

,Customer's Name,Review Rating,Rewiew Title,Review Country,Review Date,Comment,purchase Type
77,LouisianaHotsauc,1.0,Do NOT buy!!!,the United States,"December 27, 2021",Lags and freezes far too much also overheats makes up reasons not to charge itself and something weird is always happening with the camera when im live streaming or video chatting it flickers light and dark by itself its annoying and inconvenient 😑i should have waited and got what I really wanted 🙄,not varified
78,Ben,1.0,Many years later and still making junk,the United States,"October 13, 2021","I've had OnePlus devices in the past but took a break from them for awhile. I thought I'd give this phone a try being in lower cost range and that maybe OnePlus had improved over the years. Needless to say, they have not. The software is stable but it isn't anything special, I still prefer stock Android. But about the phone...\nCamera is awful even in ideal lighting. No notification light. No repeat always on notifications. No always on clock? A bit slower than I expect but that's also because it's ALWAYS closing apps and they have to reopen. I literally cannot switch from one app to another without the first closing.\n\nYou're better off getting an older Galaxy for the same price. The galaxy will be significantly better.",not varified
79,Amazon Customer,1.0,hyped up garbage,the United States,"October 3, 2021","probably the worst phone I've had. screens more brittle than glass even with a protector, the screens leds will bug out while watching videos the battery is about the same as every average phone. the feel of the phone in your hand is undesirable. being a left handed the touch print being on the right side of the phones case makes everything awkward and a pain to deal with if I need to unlock my phone while doing something. the camera is less than average quality you're better off buying a phone from 2013 if you want a decent camera on the damn thing. the only positive feature I can say about this junk is the zoom range to quality of the picture that comes from it isnt all bad. besides that not worth it at all and while I got mine in store from T-mobile my opinion remains the same about this phone as the price for google pixel 3a is significantly better and much better quality than cheap Chinese junk.",not varified
80,SB,1.0,Make you nuts.,the United States,"April 6, 2022",Never had such an intrusive pain in the ass phone. Notifications for every little darn thing Non-Stop. Just do your damn job and be a phone I sent it back,Verified
81,Afrochique,1.0,Phone was not unlocked and was not us version,the United States,"September 17, 2021",Received phone. Unable to activate it with TMobile due to phone not being compatible. Description of phone is incorrect. Was told it is not US version.,Verified


> ### Saving The Output

In [6]:
main_df.to_csv('Amazon test.csv', index = False)